In [1]:
from numpy import zeros, ones
from numpy.random import randn, randint

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU, Dropout
from tensorflow.keras.utils import plot_model

from matplotlib import pyplot as plt

Discriminator model - Input to the model - 128 x 128 x 3 images
output - binary (true/fake image)

In [2]:
def discriminator(input_shape=(128, 128, 3)):
    
    model = Sequential()
    model.add(Conv2D(128, (3, 3), padding='same', input_shape=input_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(128, (3, 3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(128, (3, 3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(128, (3, 3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(128, (3, 3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    optimizer = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model



In [3]:
# test_discrm = discriminator()
# print(test_discrm.summary())
# plot_model(test_discrm, to_file='disc_model.png', show_shapes=True)

Generator - Input to genertor network will be a latent vector. And the model will output a 128 x 128 x 3 image that can be fed into the discriminator

In [4]:
def generator(latent_dimension):
    model = Sequential()
    nodes = 128 * 8 * 8
    model.add(Dense(nodes, input_dim=latent_dimension))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((8, 8, 128)))
    model.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(3, (8, 8), activation='tanh', padding='same'))
    return model

In [5]:
# test_gen = generator(100)
# print(test_gen.summary)
# plot_model(test_gen, to_file='generator_model.png', show_shapes=True)

In [6]:
def define_gan(generator_model, discriminator_model):
    discriminator_model.trainable = False
    model = Sequential()
    model.add(generator_model)
    model.add(discriminator_model)
    optimizer = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=optimizer)
    return model

In [7]:
# test_gan = define_gan(test_gen, test_discrm)
# print(test_gan.summary())
# plot_model(test_gan, to_file='combined_moel.png', show_shapes=True)

In [8]:
#Function to sample some random real images
def generate_real_images(dataset, n_samples):
    ix = randint(0, dataset.shape[0], n_samples)
    X = dataset[ix]
    y = ones((n_samples, 1))  #Class label for real image is 1
    return X, y

In [9]:
#Function to generate random latent points
def generate_latent_points(latent_dim, n_samples):
    x_input = randn(latent_dim*n_samples)
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

In [10]:
#Function to generate fake images using latent Vectors
def generate_fake_samples(gen_model, latent_dim, n_samples):
    x_input = generate_latent_points(latent_dim, n_samples)   # Generate latent points as input to the generator
    X = gen_model.predict(x_input)   #Uses the generator model to generate fake images
    y = zeros((n_samples, 1))  # Class label for fake image is 0
    return X, y
    

In [11]:
def save_plot(examples, epoch, n=10):
    examples = (examples + 1) / 2.0 
    for i in range(n * n):
        plt.subplot(n, n, 1 + 1)
        plt.axis('off')
        plt.imshow(examples[i])
    file_name = 'generated_plot_128x128_e%03d.png' % (epoch+1)
    plt.savefig(file_name)
    plt.close()

In [12]:
#Funciton to summarize the performace periodically
def summarize_performace(epoch, gen_model, disc_model, dataset, latent_dim, n_samples=100):
    #Lets fetch the real images
    X_real, y_real = generate_real_images(dataset, n_samples)
    _, acc_real = disc_model.evaluate(X_real, y_real, verbose=0)
    x_fake, y_fake = generate_fake_samples(gen_model, latent_dim, n_samples)
    _, acc_fake = disc_model.evaluate(x_fake, y_fake, verbose=0)
    print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
    save_plot(x_fake, epoch)
    filename = 'generator_model_128x128_%03d.h5' % (epoch+1)
    gen_model.save(filename)
    

In [13]:
def train(gen_model, disc_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=128):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)  # Discriminator trained on half real and half fake images
    #Enumerate epochs
    for i in range(n_epochs):
        #enumerate batches
        for j in range(bat_per_epo):
            X_real, y_real = generate_real_images(dataset, half_batch)  #Fetch real images
            d_loss1, _ = disc_model.train_on_batch(X_real, y_real)       #Train the discriminator using real images
            X_fake, y_fake = generate_fake_samples(gen_model, latent_dim, half_batch)   #Generate fake images
            d_loss2, _ = disc_model.train_on_batch(X_fake, y_fake)    #Train discriminator on fake images
            X_gan = generate_latent_points(latent_dim, n_batch)       #Generate latent vectors as input for generator
            y_gan = ones((n_batch, 1))      #Label generates (fake) images as 1 to fool the discriminator
            g_loss = gan_model.train_on_batch(X_gan, y_gan)   # Train the generator (via the discriminator error)
            print('Epoch>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %(i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))   #Reports generator and discriminator errors
        if (i+1) % 20 == 0:
            summarize_performace(i, gen_model, disc_model, dataset, latent_dim)
            

In [14]:
import os
import numpy as np
import cv2
from PIL import Image
import random

In [15]:
import numpy

In [16]:
n=15000   #Number of images to be read form the directory
SIZE = 128   #Resize the images to this size
all_img_list = os.listdir('../input/xai-imagedata/UTKFace/')   

dataset_list = random.sample(all_img_list, n)   #Get 20,000 random images from the directory

dataset = []


for img in dataset_list:
    temp_img  = cv2.imread('../input/xai-imagedata/UTKFace/'+img)
    temp_img = cv2.cvtColor(temp_img, cv2.COLOR_BGR2RGB)
    temp_img = Image.fromarray(temp_img)
    temp_img = temp_img.resize((SIZE, SIZE))
    dataset.append(np.array(temp_img))
    
dataset = np.array(dataset)  #Convert the list to numpy array

dataset = dataset.astype(numpy.float64)

dataset = (dataset - 127.5) / 127.5

latent_dim = 100

#Create the discriminator model using our pre-defined function
disc_model = discriminator()
#Create the generator using our predefined finction
gen_model = generator(latent_dim)
#create GAN using generator and discriminator
gan_model = define_gan(gen_model, disc_model)

train(gen_model, disc_model, gan_model, dataset, latent_dim, n_epochs=100)




2022-07-02 20:52:59.593424: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-02 20:52:59.693924: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-02 20:52:59.694628: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-02 20:52:59.697479: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Epoch>1, 1/117, d1=0.709, d2=0.695 g=0.692
Epoch>1, 2/117, d1=0.638, d2=0.695 g=0.692
Epoch>1, 3/117, d1=0.541, d2=0.699 g=0.689
Epoch>1, 4/117, d1=0.375, d2=0.713 g=0.677
Epoch>1, 5/117, d1=0.183, d2=0.760 g=0.644
Epoch>1, 6/117, d1=0.076, d2=0.866 g=0.601
Epoch>1, 7/117, d1=0.038, d2=0.949 g=0.598
Epoch>1, 8/117, d1=0.080, d2=0.926 g=0.643
Epoch>1, 9/117, d1=0.108, d2=0.781 g=0.736
Epoch>1, 10/117, d1=0.121, d2=0.636 g=0.836
Epoch>1, 11/117, d1=0.067, d2=0.575 g=0.947
Epoch>1, 12/117, d1=0.021, d2=0.537 g=1.025
Epoch>1, 13/117, d1=0.037, d2=0.531 g=1.007
Epoch>1, 14/117, d1=0.024, d2=0.535 g=0.975
Epoch>1, 15/117, d1=0.013, d2=0.525 g=0.989
Epoch>1, 16/117, d1=0.018, d2=0.495 g=1.047
Epoch>1, 17/117, d1=0.010, d2=0.447 g=1.181
Epoch>1, 18/117, d1=0.003, d2=0.381 g=1.409
Epoch>1, 19/117, d1=0.029, d2=0.338 g=1.701
Epoch>1, 20/117, d1=0.000, d2=0.228 g=2.472
Epoch>1, 21/117, d1=0.000, d2=0.070 g=3.593
Epoch>1, 22/117, d1=0.000, d2=0.032 g=4.030
Epoch>1, 23/117, d1=0.000, d2=0.027 g=4.1